Firstly, let's consider a simple tensor and do all the procedures for it to visualize the process.

In [27]:
# Write a function for generating random tensor with given shape

In [2]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

In [91]:
def lhs(Nl, Nr, r, s): # some problem occurs here
    if(Nl > Nr):
        m = Nr
    else:
        m = Nl
    summ = 0
    for k in range(r+1, m):
        summ += s[k]**2
    return np.sqrt(np.max(summ))

In [92]:
def compar(lhs, rhs):
    if(lhs <= rhs):
        return True
    else:
        return False

In [93]:
def truncated_SVD(Nl, Nr, r, u, s, vh, eps, rhs): 
    l = lhs(Nl, Nr, r, s)
    while(compar(l, rhs) == False):
        u = u[:, :-1]
        s = s[:-1, :-1]
        vh = vh[:-1, :]
    return u, s, vh

In [98]:
# now, we have a tensor with, most likely, full-rank (since it generated randomly)

eps = 1
d = 4

A = np.random.rand(4, 4, 4, 4)
n = A.shape

# ===================1=====================

Nl = n[0]
Nr = np.prod(n[1:])

# ===================2=====================

M = A.reshape((Nl, Nr))

# ===================3=====================

nrm = la.norm(M, ord='fro')
rhs = (eps*nrm) / (np.sqrt(d-1))

# ===================4=====================

u, s, vh = np.linalg.svd(M, full_matrices=True, compute_uv=True)
# vh - unitary rows (already transposed), u - unitary columns
aux = np.zeros((M.shape[0], M.shape[1]))
# concatenate zeros to the tail of singular values array (they should have correct shapes)
for i in range(s.shape[0]):
    aux[i, i] = s[i]
s = aux

# ===================5=====================

u, s, vh = truncated_SVD(Nl, Nr, 2, u, s, vh, eps, rhs)

# ===================6=====================

G = list()

G.append(u.reshape(1, u.shape[0], u.shape[1]))
M = s @ vh
r = np.count_nonzero(s)

# ===================7=====================





# MAIN LOOP





# ===================7=====================

for k in range(2, d):
    Nl = n[k]
    Nr = Nr // n[k]
    
    M = M.reshape((r*Nl, Nr))
    aux = np.zeros((M.shape[0], M.shape[1]))
    
    u, s, vh = np.linalg.svd(M, full_matrices=True, compute_uv=True)

    for i in range(s.shape[0]):
        aux[i, i] = s[i]
    s = aux # now s has correct shape to compute the inner product
    
    u, s, vh = truncated_SVD(Nl, Nr, s.shape[0], u, s, vh, eps, rhs)

    r_prev = r
    r = np.count_nonzero(s)
    
    # cut all the zero rows and columns of s and corresponding rows and columns of matrices U and V^T
    u = u[:, :r]
    s = s[:r, :r]
    vh = vh[:r, :]
    
    G.append(u.reshape(r_prev, n[k], r))
    
    M = s @ vh

print('r = ', r)
G.append(M.reshape(r, n[d-1], 1))
print("CARRIAGES")
for i in range(d):
    print(G[i].shape)

r =  4
CARRIAGES
(1, 4, 4)
(4, 4, 16)
(16, 4, 4)
(4, 4, 1)


In [99]:
G[2].shape

(16, 4, 4)

In [100]:
Nr = Nr // n[k]
Nr

1

In [101]:
r

4

In [102]:
a = np.array([[1, 2, 3, 4], 
              [1, 2, 3, 4]])
a[:, :-2]

array([[1, 2],
       [1, 2]])

In [103]:
from tt import * 
import maxvolpy as mv

K = np.array([[2, 3, 4, 5, 6, 7], # a string
              [10, 3, 45, 5, 6, 1], 
              [2, 30000, 4, 4, 40, 7], 
              [2, 3, 6, 567, 6, 7], 
              [2, 3, 4, 5, 6, 20], 
             [2, 3, 4, 5, 6, 20], 
             [2, 3, 4, 4, 6, 20], 
             [2, 3, 4, 5, 7, 20], 
             [2, 3, 9, 5, 6, 20]])
print(K.shape)

f, _ = mv.maxvol.maxvol_qr(K, tol=1)
print(f)

K1 = K[f, :]
K1

(9, 6)
[1 2 0 3 8 5]


array([[   10,     3,    45,     5,     6,     1],
       [    2, 30000,     4,     4,    40,     7],
       [    2,     3,     4,     5,     6,     7],
       [    2,     3,     6,   567,     6,     7],
       [    2,     3,     9,     5,     6,    20],
       [    2,     3,     4,     5,     6,    20]])

In [104]:
# Let's check the results

In [106]:
G1 = np.tensordot(G[0], G[1], axes=1) 
print(G[2].shape)
G2 = np.tensordot(G1, G[2], axes=1)
G3 = np.tensordot(G2, G[3], axes=1)
G3 = G3.reshape(4, 4, 4, 4, order='f')
G3.shape
if(np.linalg.norm(G3 - A) < 1e-3):
    print('ok')

(16, 4, 4)
ok


### TT cross approximation (function)

In [87]:
# INPUT: 
    # T - initial tensor
    # eps - accuracy bound of the approximation
    # n - array with length of each dimension
    # method - either SVD or maxvol

# OUTPUT:
    # G - list of 'carriages' of the train




def TT(A, eps=1e-3, method='maxvol'):
    def lhs(Nl, Nr, r, s):
        print('Parameters in lhs function: s = {} \n Nl = {} \n Nr = {}, \n r = {}'.format(s.shape, Nl, Nr, r))
        
        if(Nl > Nr):
            m = Nr
        else:
            m = Nl
        summ = 0
        for k in range(r+1, m):
            summ += s[k]**2
            print('summ = {}'.format(summ))
        return np.sqrt(summ)
    def compar(lhs, rhs):
        if(lhs <= rhs):
            return True
        else:
            return False
    def truncated_SVD(Nl, Nr, r, u, s, vh, eps): 
        print('I`m here!')
        print('rhs= {}'.format(rhs))
        l = lhs(Nl, Nr, r, s)
        print('lhs= {}'.format(l))
        
        while(compar(l, rhs) == False):
            u = u[:, :-1]
            s = s[:-1]
            vh = vh[:-1, :]
        return u, s, vh
#     def cross():

    n = A.shape
    d = len(n) # number of dimensions

    Nl = n[0]
    Nr = np.prod(n[1:])
    print('Nl: {}'.format(Nl))
    print('Nr: {}'.format(Nr))

    M = A.reshape((Nl, Nr))
    print('shape(M)', M.shape)
    
    if(method == 'SVD'):
        print('INITIAL ITERATION: ')
        nrm = la.norm(M, ord='fro')
        rhs = (eps*nrm) / (np.sqrt(d-1))
    
        u, s, vh = np.linalg.svd(M, full_matrices=True, compute_uv=True)
        print('shape(u) = ', u.shape)
        print('shape(s) = ', s.shape)
        print('shape(vh) = ', vh.shape)

        
#         print('shape of s after fitting the diag into a matrix', s.shape)
#         print('get back to unfolding ', (u@s@vh).shape)
#         # check:
#         if((u@s@vh).all() == M.all()):
#             print('deconposition is correct')

        u, s, vh = truncated_SVD(Nl, Nr, s.shape[0], u, s, vh, eps)
#         print('shape(u_trunceted) = ', u.shape)
#         print('shape(s_trunc) = ', s.shape)
#         print('shape(vh_trunc) = ', vh.shape)

        G = list()
        G.append(u.reshape(1, u.shape[0], u.shape[1]))
        
        aux = np.zeros((M.shape[0], M.shape[1]))
        for i in range(s.shape[0]):
            aux[i, i] = s[i]
        s = aux
        
        M = s @ vh
        r = np.count_nonzero(s)

        for k in range(1, d-1):
            print('ITERATION {}'.format(k))
            Nl = n[k]
            Nr = Nr // n[k]

            M = M.reshape((r*Nl, Nr))

            u, s, vh = np.linalg.svd(M, full_matrices=True, compute_uv=True)
            
            print('Sing. val. shape from original SVD {}'.format(s.shape))

#             u, s, vh = truncated_SVD(Nl, Nr, s.shape[0], u, s, vh, eps)

            r_prev = r
            r = s.shape[0]

            u = u[:, :r]
            s = s[:r]
            vh = vh[:r, :]

            G.append(u.reshape(r_prev, n[k], r))
            
            aux = np.zeros((M.shape[0], M.shape[1]))
            for i in range(s.shape[0]):
                aux[i, i] = s[i]
            s = aux 
            
            M = s @ vh
                
        G.append(M.reshape(r, n[d-1], 1))
        
        print("CARRIAGES")
        for i in range(d):
            print(G[i].shape)
            
        return G
        
    if(method == 'maxvol'):
        return None
    
def test(A, G, eps):
    G1 = np.tensordot(G[0], G[1], axes=1)
    for i in range(len(B.shape)-2):
        G1 = np.tensordot(G1, G[i+2], axes=1) 

    G1 = G1.reshape(B.shape)

    if(np.linalg.norm(G1 - B) < 1e-3):
        print('ok')
        return 0   

In [88]:
B = np.random.rand(10, 20, 13, 5, 3)
G = TT(B, eps=1e-3, method='SVD')
# here some problems with truncation

Nl: 10
Nr: 3900
shape(M) (10, 3900)
INITIAL ITERATION: 
shape(u) =  (10, 10)
shape(s) =  (10,)
shape(vh) =  (3900, 3900)
I`m here!
rhs= 0.05683478466559357
Parameters in lhs function: s = (10,) 
 Nl = 10 
 Nr = 3900, 
 r = 10
lhs= 0.0
ITERATION 1
Sing. val. shape from original SVD (195,)
ITERATION 2


ValueError: cannot reshape array of size 39000 into shape (2535,15)

In [25]:
test(B, G, eps=1e-3)

ok


In [230]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

eps = 1e-2

d = 4

A = np.array(
[[[  [1, 8, 3], 
    [1, 2, 4], 
    [1, 3, 5]   ],
[   [1, 8, 3], 
    [1, 2, 4], 
    [1, 7, 5]   ],
[   [1, 2, 9], 
    [1, 5, 4], 
    [1, 3, 5]   ], 
[   [8, 2, 3], 
    [1, 4, 4], 
    [1, 3, 5]   ]], 
[[  [1, 8, 3], 
    [1, 2, 4], 
    [1, 3, 5]   ],
[   [1, 8, 3], 
    [1, 2, 4], 
    [1, 3, 9]   ],
[   [1, 2, 3], 
    [1, 5, 4], 
    [1, 3, 5]   ], 
[   [6, 2, 3], 
    [1, 4, 1], 
    [1, 3, 5]   ]],
[[  [1, 8, 3], 
    [1, 2, 2], 
    [1, 3, 5]   ],
[   [1, 8, 3], 
    [1, 2, 6], 
    [1, 3, 5]   ],
[   [1, 2, 3], 
    [1, 5, 4], 
    [1, 3, 5]   ], 
[   [1, 2, 3], 
    [1, 4, 4], 
    [1, 3, 5]   ]]]    )

n = A.shape

In [176]:
n

(3, 4, 3, 3)

In [210]:
G1 = np.tensordot(G[0], G[1], axes=1) 
print(G[2].shape)
G2 = np.tensordot(G1, G[2], axes=1)
G3 = np.tensordot(G2, G[3], axes=1)
G3 = G3.reshape(3, 4, 3, 3)
G3.shape
if(np.linalg.norm(G3 - A) < 1e-3):
    print('ok')

(9, 3, 3)
ok


In [68]:

A = np.random.rand(3, 3, 3, 3)
n1 = A.shape

tuple